# Global Vaccination Effort 

**This notebook seeks to explore the latest update of the coronavirus vaccine using visualizations. **

![](https://i1.wp.com/sitn.hms.harvard.edu/wp-content/uploads/2018/09/07-blog-Vaccines.jpg?resize=1200%2C630)

*Last Notebook Update: 5/13/2021 10:29 PM ET. If data is not available for a specific field, it will appear as 0. *

**Feel free to provide me with feedbacks **

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import plotly.express as px
import geopandas as gpd
plt.style.use('seaborn-bright') 

In [ ]:
data = pd.read_csv("/kaggle/input/covid-world-vaccination-progress/country_vaccinations.csv")
data.head()

In [ ]:
data.isnull().sum()

In [ ]:
dates = data.date.unique();
dates.sort()
latest_date = dates[-1];

In [ ]:
# latest_data = data[data.date==latest_date]

data.sort_values(by=['total_vaccinations'], ascending=False, inplace=True);

# drop all the null values in the data we are using 

# latest_data.dropna(subset=['total_vaccinations', 'vaccines', 'daily_vaccinations', 'people_vaccinated_per_hundred'], inplace=True)


countries = data.country.unique()

vaccine_countries = []
total_vaccination = [] 
vaccine_name = [] 
daily_vaccination = [] 
people_vaccinated_per_hundred = [] 
iso_codes = [] 
latest_data_update = [] 
people_fully_vaccinated = [] 
people_fully_vaccinated_per_hundred  = []


# getting the latest vaccine data possible for each country 

for i in countries: 
    for d in dates[::-1]:
        total = data[data.country==i]['total_vaccinations'][data.date==d].sum()
        name = data[data.country==i]['vaccines'][data.date==d].sum()
        daily = data[data.country==i]['daily_vaccinations'][data.date==d].sum()
        per_hundred = data[data.country==i]['people_vaccinated_per_hundred'][data.date==d].sum(); 
        iso = data[data.country==i]['iso_code'][data.date==d].sum();
        fully = data[data.country==i]['people_fully_vaccinated'][data.date==d].sum(); 
        fully_per_hundred = data[data.country==i]['people_fully_vaccinated_per_hundred'][data.date==d].sum(); 
        
        # if one of those features are null
        if(total == 0):
            pass
        else:
            vaccine_countries.append(i)
            total_vaccination.append(total)
            vaccine_name.append(name) 
            
            daily_vaccination.append(daily)
            people_vaccinated_per_hundred.append(per_hundred)
            
            people_fully_vaccinated.append(fully)
            people_fully_vaccinated_per_hundred.append(fully_per_hundred);
            
            iso_codes.append(iso)
            latest_data_update.append(d)
            break

In [ ]:
print("Vaccine country length:", len(vaccine_countries))
print("Total vaccinaton length:", len(total_vaccination))
print("Vaccine name length:", len(vaccine_name))
print("Daily vaccinaton length:", len(daily_vaccination))
print("People vaccinated per hundred length:", len(people_vaccinated_per_hundred))
print("ISO codes length:", len(iso_codes))

# Data Table on the Global Vaccine Effort

***if a value is 0, then it is N/A**

In [ ]:
latest_vaccine_data = pd.DataFrame({'Country Name': vaccine_countries, 'Total Vaccinations': [format(int(i), ',d') for i in total_vaccination],
                          'Daily Vaccination': [format(int(i), ',d') for i in daily_vaccination], 'People Vaccinated per Hundred' : people_vaccinated_per_hundred, 
                          'Total Fully Vaccinations' : [format(int(i), ',d') for i in  people_fully_vaccinated], 'People Fully Vaccinated per Hundred': people_fully_vaccinated_per_hundred,
                          'Vaccine Name' : vaccine_name, 'Latest Data Update' : latest_data_update})

latest_vaccine_data.style.background_gradient(cmap='Greens')

# Displaying the data using a bar chart
this shows the top 15 countries for each respective categories

In [ ]:
import operator
def plot_bar_graphs(x, y, title, x_title):
    plt.figure(figsize=(16,10))
    plt.barh(x, y)
    plt.title(title, size=20)
    plt.xticks(size=20)
    plt.yticks(size=18)
    plt.xlabel(x_title, size=20)
    plt.show()
    

# sort each feature in a descending manner       
vaccine_countries_total = [[k, v] for k, v in sorted(zip(vaccine_countries, total_vaccination), key=operator.itemgetter(1), reverse=True)]
vaccine_countries_daily = [[k, v] for k, v in sorted(zip(vaccine_countries, daily_vaccination), key=operator.itemgetter(1), reverse=True)]
vaccine_countries_per_hundred = [[k, v] for k, v in sorted(zip(vaccine_countries, people_vaccinated_per_hundred), key=operator.itemgetter(1), reverse=True)]
vaccine_countries_fully = [[k, v] for k, v in sorted(zip(vaccine_countries, people_fully_vaccinated), key=operator.itemgetter(1), reverse=True)]
vaccine_countries_fully_per_hundred = [[k, v] for k, v in sorted(zip(vaccine_countries, people_fully_vaccinated_per_hundred), key=operator.itemgetter(1), reverse=True)]
    
plot_bar_graphs([i[0] for i in vaccine_countries_total][:15], [i[1] for i in vaccine_countries_total][:15], 'Total Vaccination', 'Number of Vaccinations')
plot_bar_graphs([i[0] for i in vaccine_countries_daily][:15], [i[1] for i in vaccine_countries_daily][:15], 'Daily Vaccination', 'Number of Vaccinations')
plot_bar_graphs([i[0] for i in vaccine_countries_per_hundred][:15],[i[1] for i in vaccine_countries_per_hundred][:15], 'People Vaccinated per Hundred'.format(latest_date), 'Number of Vaccinations')
plot_bar_graphs([i[0] for i in vaccine_countries_fully][:15],[i[1] for i in  vaccine_countries_fully][:15], 'People Fully Vaccinated'.format(latest_date), 'Number of Vaccinations')
plot_bar_graphs([i[0] for i in vaccine_countries_fully_per_hundred][:15],[i[1] for i in vaccine_countries_fully_per_hundred][:15], 'People Fully Vaccinated per Hundred'.format(latest_date), 'Number of Vaccinations')

# Geolocation Vaccine Data

**Some countries have missing data for the number of fully vaccinations and people vaccinated per hundred, it will simply appear as 0. **

In [ ]:
fig = px.scatter_geo(latest_vaccine_data, locations=iso_codes, color=total_vaccination,
                     hover_name="Country Name", size=[i for i in total_vaccination],
                     projection="natural earth", title="Total Vaccination")
fig.show()

In [ ]:
# fig = px.choropleth(latest_vaccine_data, locations=iso_codes, color='Vaccine Name',
#                      hover_name="Country Name",
#                      projection="natural earth", title="Total Vaccinaton by Vaccine Name")
# fig.show()

In [ ]:
fig = px.scatter_geo(latest_vaccine_data, locations=iso_codes, color=daily_vaccination,
                     hover_name="Country Name", size=[i for i in daily_vaccination],
                     projection="natural earth", title="Daily Vaccination")
fig.show()

In [ ]:
fig = px.scatter_geo(latest_vaccine_data, locations=iso_codes, color='People Vaccinated per Hundred',
                     hover_name="Country Name", size="People Vaccinated per Hundred",
                     projection="natural earth", title="People Vaccinated per Hundred")
fig.show()

In [ ]:
fig = px.scatter_geo(latest_vaccine_data, locations=iso_codes, color=[i for i in people_fully_vaccinated],
                     hover_name="Country Name", size=[i for i in people_fully_vaccinated],
                     projection="natural earth", title="Total Fully Vaccinations")
fig.show()

In [ ]:
fig = px.scatter_geo(latest_vaccine_data, locations=iso_codes, color='People Fully Vaccinated per Hundred',
                     hover_name="Country Name", size="People Fully Vaccinated per Hundred",
                     projection="natural earth", title="People Fully Vaccinated per Hundred")
fig.show()